# Setup

Instalacja AutoSklearn

In [ ]:
# 1. uninstall all affected packages
!pip uninstall -y Cython scipy pyparsing scikit_learn imbalanced-learn mlxtend yellowbrick
# 2. install packages to be downgraded
!pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4
# 3. install older scikit-learn disregarding its dependencies
!pip install scikit-learn==0.24.2 --no-build-isolation
# 4. finally install auto-sklearn
!pip install auto-sklearn
# 5. then, try load

Instalacja AutoGluon

In [ ]:
#!pip install autogluon
!pip install -U pip
!pip install -U setuptools wheel

!pip install torch==2.0.1+cpu torchvision==0.15.2+cpu --index-url https://download.pytorch.org/whl/cpu

!pip install autogluon

Teraz należy uruchomić ponownie sesję.

## Załadowanie danych i określenie funkcji scoringowej

In [7]:
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import pandas as pd
pd.set_option('display.width', 120)

df = pd.read_csv("https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data", sep = " ", header = None)
y = pd.read_csv("https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels", header = None)
df = df.drop(500, axis = 1)
df["y"] = y
X = df.drop("y", axis = 1)
X_final_test = pd.read_csv("https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data", sep = " ", header = None)
X_final_test = X_final_test.drop(500, axis = 1)

def get_data_scoring_function_and_predictions(X, y, X_final_test):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y, test_size = 0.2, random_state = 2023)
    def scoring_function(estimator):
        predictions = estimator.predict(X_test)
        return sklearn.metrics.balanced_accuracy_score(y_test, predictions)

    def train_scoring_function(estimator):
        predictions = estimator.predict(X_train)
        return sklearn.metrics.balanced_accuracy_score(y_train, predictions)

    def predictions(estimator):
        predictions_probs = estimator.predict_proba(X_final_test)[:,1]
        return predictions_probs

    return X_train, y_train, X_test, y_test, scoring_function, train_scoring_function, predictions

X_train, y_train, X_test, y_test, scoring_function, train_scoring_function, predictions = get_data_scoring_function_and_predictions(X, y, X_final_test)

# AutoSklearn 1.0 i AutoSklearn 2.0

AutoSklearn 1.0

In [8]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.metrics import balanced_accuracy

In [9]:
settings = {
  "time_left_for_this_task": 3600,
  "seed": 42,
  "metric": balanced_accuracy,
  "n_jobs": 4,
}

resampling_strategy = "holdout"


In [ ]:
askl1 = AutoSklearnClassifier(
    **settings,
    resampling_strategy=resampling_strategy
)
askl1.fit(X_train, y_train, dataset_name="artificial_train")

In [13]:
from pprint import pprint

print(f"Auto-sklearn 1.0 | train = {train_scoring_function(askl1)} | test = {scoring_function(askl1)}")
print(f"Selected `resampling-strategy` = {askl1.resampling_strategy}")
print(f"Selected `resampling-strategy-arguments` = {askl1.resampling_strategy_arguments}")

print(askl1.sprint_statistics())

leaderboard = askl1.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)

pprint(askl1.show_models())

Auto-sklearn 1.0 | train = 0.9587247420296376 | test = 0.8524102410241023
Selected `resampling-strategy` = holdout
Selected `resampling-strategy-arguments` = None
auto-sklearn results:
  Dataset name: artificial_train
  Metric: balanced_accuracy
  Best validation score: 0.871167
  Number of target algorithm runs: 214
  Number of successful target algorithm runs: 167
  Number of crashed target algorithm runs: 32
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 14

          rank  ensemble_weight           type      cost   duration
model_id                                                           
7           16             0.04  random_forest  0.153519  69.785877
89          11             0.04  random_forest  0.144071  59.766010
103         17             0.06  random_forest  0.168771  21.749009
151          7             0.02  random_forest  0.144028  58.788337
155          9             0.04  random_forest  0

AutoSklearn 2.0

In [ ]:
askl2 = AutoSklearn2Classifier(**settings)
askl2.fit(X_train, y_train, dataset_name="artificial_train")

In [7]:
from pprint import pprint

print(f"Auto-sklearn 2.0 | train = {train_scoring_function(askl2)} | test = {scoring_function(askl2)}")
print(f"Selected `resampling-strategy` = {askl2.resampling_strategy}")
print(f"Selected `resampling-strategy-arguments` = {askl2.resampling_strategy_arguments}")

print(askl2.sprint_statistics())

leaderboard = askl2.leaderboard(sort_by="model_id", ensemble_only=True)
print(leaderboard)
pprint(askl2.show_models())

Auto-sklearn 2.0 | train = 0.9993734335839599 | test = 0.8400840084008401
Selected `resampling-strategy` = cv-iterative-fit
Selected `resampling-strategy-arguments` = {'folds': 10}
auto-sklearn results:
  Dataset name: artificial_train
  Metric: balanced_accuracy
  Best validation score: 0.838792
  Number of target algorithm runs: 58
  Number of successful target algorithm runs: 57
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

          rank  ensemble_weight                type      cost     duration
model_id                                                                  
2            3             0.02         extra_trees  0.188154   114.713903
8            2             0.02         extra_trees  0.178153   121.770025
9            5             0.86                 sgd  0.458132    42.593864
15           1             0.04       random_forest  0.161208  1440.

# AutoGluon

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
df_train = X_train
df_train["y"] = y_train
df_test = X_test
df_test["y"] = y_test

In [4]:
label = "y"
time_limit = 3600
metric = 'balanced_accuracy'
predictor = TabularPredictor(label, eval_metric=metric).fit(df_train, time_limit=time_limit, presets='best_quality')
predictor.leaderboard(df_test)

No path specified. Models will be saved in: "AutogluonModels/ag-20240116_135906"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240116_135906/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20240116_135906/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.842659,0.861851,balanced_accuracy,0.123305,0.182982,340.949994,0.123305,0.182982,340.949994,1,True,7
1,WeightedEnsemble_L2,0.842659,0.863104,balanced_accuracy,0.690010,1.008118,573.810237,0.004045,0.013373,4.735965,2,True,21
2,LightGBM_r131_BAG_L1,0.842509,0.851230,balanced_accuracy,0.111198,0.085011,143.630250,0.111198,0.085011,143.630250,1,True,16
3,CatBoost_r177_BAG_L1,0.837709,0.853749,balanced_accuracy,0.124973,0.198139,333.920694,0.124973,0.198139,333.920694,1,True,14
4,LightGBM_BAG_L1,0.832458,0.835002,balanced_accuracy,0.147030,0.076413,140.540159,0.147030,0.076413,140.540159,1,True,4
5,LightGBMLarge_BAG_L1,0.830033,0.836863,balanced_accuracy,0.159840,0.092669,336.427042,0.159840,0.092669,336.427042,1,True,13
6,XGBoost_BAG_L1,0.825033,0.829360,balanced_accuracy,0.221251,0.155648,115.961633,0.221251,0.155648,115.961633,1,True,11
7,LightGBMXT_BAG_L1,0.778053,0.800566,balanced_accuracy,0.183284,0.103673,74.159168,0.183284,0.103673,74.159168,1,True,3
8,CatBoost_r9_BAG_L1,0.777953,0.784917,balanced_accuracy,0.160508,0.359867,648.725132,0.160508,0.359867,648.725132,1,True,18
9,LightGBM_r96_BAG_L1,0.730473,0.744895,balanced_accuracy,0.122968,0.106899,55.947199,0.122968,0.106899,55.947199,1,True,19


In [6]:
print(f"AutoGluon | train = {train_scoring_function(predictor)} | test = {scoring_function(predictor)}")

AutoGluon | train = 0.9893671210445065 | test = 0.8426592659265926


# Zapisanie predykcji dla najlepszego modelu

Najlepszym modelem okazał się ten na podstawie AutoSklearn 1.0.

In [14]:
pd.Series(
    predictions(askl1),
    name="306013_306019"
).to_csv("306013_306019_artifical_automl_prediction.txt", index=False)